# Collaborative Filtering

## Import

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Dataset Representation

In [23]:
book_df = pd.read_csv('Dataset3.csv', index_col=0)
book_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,1.0,5.0,4.0,NaN,1.0,3.0,4.0,1.0,5.0,NaN,...,3.0,5.0,NaN,2.0,NaN,1.0,4.0,5.0,3.0,4.0
1,NaN,NaN,5.0,3.0,4.0,1.0,4.0,NaN,NaN,1.0,...,5.0,4.0,2.0,1.0,1.0,1.0,2.0,3.0,5.0,4.0
2,2.0,4.0,1.0,NaN,NaN,4.0,3.0,NaN,4.0,4.0,...,3.0,2.0,4.0,NaN,3.0,2.0,4.0,5.0,1.0,4.0
3,3.0,5.0,1.0,5.0,1.0,3.0,5.0,3.0,2.0,4.0,...,1.0,2.0,2.0,2.0,5.0,4.0,NaN,NaN,3.0,NaN
4,NaN,NaN,5.0,3.0,2.0,1.0,4.0,NaN,4.0,1.0,...,3.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.0,5.0,5.0,2.0,NaN,NaN,3.0,2.0,4.0,5.0,...,4.0,1.0,NaN,5.0,NaN,2.0,5.0,4.0,3.0,4.0
96,3.0,4.0,NaN,3.0,NaN,3.0,NaN,1.0,3.0,2.0,...,4.0,3.0,1.0,4.0,5.0,5.0,3.0,2.0,4.0,NaN
97,2.0,2.0,2.0,NaN,4.0,1.0,1.0,4.0,5.0,3.0,...,4.0,4.0,5.0,2.0,5.0,4.0,2.0,NaN,4.0,4.0
98,1.0,5.0,NaN,NaN,1.0,5.0,5.0,4.0,4.0,NaN,...,3.0,2.0,5.0,4.0,NaN,3.0,3.0,5.0,3.0,2.0


In [24]:
col_names = []
row_names = []

for i in range(100):
    row_names.append("Book " + str(i))

for i in range(300):
    col_names.append("User " + str(i))
    
book_df.index = row_names
book_df.columns = col_names

book_df

,User 0,User 1,User 2,User 3,User 4,User 5,User 6,User 7,User 8,User 9,...,User 290,User 291,User 292,User 293,User 294,User 295,User 296,User 297,User 298,User 299
Book 0,1.0,5.0,4.0,NaN,1.0,3.0,4.0,1.0,5.0,NaN,...,3.0,5.0,NaN,2.0,NaN,1.0,4.0,5.0,3.0,4.0
Book 1,NaN,NaN,5.0,3.0,4.0,1.0,4.0,NaN,NaN,1.0,...,5.0,4.0,2.0,1.0,1.0,1.0,2.0,3.0,5.0,4.0
Book 2,2.0,4.0,1.0,NaN,NaN,4.0,3.0,NaN,4.0,4.0,...,3.0,2.0,4.0,NaN,3.0,2.0,4.0,5.0,1.0,4.0
Book 3,3.0,5.0,1.0,5.0,1.0,3.0,5.0,3.0,2.0,4.0,...,1.0,2.0,2.0,2.0,5.0,4.0,NaN,NaN,3.0,NaN
Book 4,NaN,NaN,5.0,3.0,2.0,1.0,4.0,NaN,4.0,1.0,...,3.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Book 95,1.0,5.0,5.0,2.0,NaN,NaN,3.0,2.0,4.0,5.0,...,4.0,1.0,NaN,5.0,NaN,2.0,5.0,4.0,3.0,4.0
Book 96,3.0,4.0,NaN,3.0,NaN,3.0,NaN,1.0,3.0,2.0,...,4.0,3.0,1.0,4.0,5.0,5.0,3.0,2.0,4.0,NaN
Book 97,2.0,2.0,2.0,NaN,4.0,1.0,1.0,4.0,5.0,3.0,...,4.0,4.0,5.0,2.0,5.0,4.0,2.0,NaN,4.0,4.0
Book 98,1.0,5.0,NaN,NaN,1.0,5.0,5.0,4.0,4.0,NaN,...,3.0,2.0,5.0,4.0,NaN,3.0,3.0,5.0,3.0,2.0


## Exploratory Data Analysis

## Data Mining


In [25]:
from collaborative_filtering import CollaborativeFiltering

In [26]:
cfilter = CollaborativeFiltering(5)

In [27]:
mean = cfilter.get_row_mean(book_df)
mean

Book 0     2.983402
Book 1     2.786008
Book 2     2.983806
Book 3     3.011905
Book 4     2.820408
             ...   
Book 95    3.112500
Book 96    2.892430
Book 97    2.919831
Book 98    2.971311
Book 99    2.887550
Length: 100, dtype: float64

In [28]:
normalized_df = cfilter.normalize_data(book_df, mean)
normalized_df.round(2)

,User 0,User 1,User 2,User 3,User 4,User 5,User 6,User 7,User 8,User 9,...,User 290,User 291,User 292,User 293,User 294,User 295,User 296,User 297,User 298,User 299
Book 0,-1.98,2.02,1.02,NaN,-1.98,0.02,1.02,-1.98,2.02,NaN,...,0.02,2.02,NaN,-0.98,NaN,-1.98,1.02,2.02,0.02,1.02
Book 1,NaN,NaN,2.21,0.21,1.21,-1.79,1.21,NaN,NaN,-1.79,...,2.21,1.21,-0.79,-1.79,-1.79,-1.79,-0.79,0.21,2.21,1.21
Book 2,-0.98,1.02,-1.98,NaN,NaN,1.02,0.02,NaN,1.02,1.02,...,0.02,-0.98,1.02,NaN,0.02,-0.98,1.02,2.02,-1.98,1.02
Book 3,-0.01,1.99,-2.01,1.99,-2.01,-0.01,1.99,-0.01,-1.01,0.99,...,-2.01,-1.01,-1.01,-1.01,1.99,0.99,NaN,NaN,-0.01,NaN
Book 4,NaN,NaN,2.18,0.18,-0.82,-1.82,1.18,NaN,1.18,-1.82,...,0.18,-1.82,-1.82,-0.82,-0.82,-0.82,-1.82,-0.82,NaN,2.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Book 95,-2.11,1.89,1.89,-1.11,NaN,NaN,-0.11,-1.11,0.89,1.89,...,0.89,-2.11,NaN,1.89,NaN,-1.11,1.89,0.89,-0.11,0.89
Book 96,0.11,1.11,NaN,0.11,NaN,0.11,NaN,-1.89,0.11,-0.89,...,1.11,0.11,-1.89,1.11,2.11,2.11,0.11,-0.89,1.11,NaN
Book 97,-0.92,-0.92,-0.92,NaN,1.08,-1.92,-1.92,1.08,2.08,0.08,...,1.08,1.08,2.08,-0.92,2.08,1.08,-0.92,NaN,1.08,1.08
Book 98,-1.97,2.03,NaN,NaN,-1.97,2.03,2.03,1.03,1.03,NaN,...,0.03,-0.97,2.03,1.03,NaN,0.03,0.03,2.03,0.03,-0.97


In [29]:
book0 = book_df.iloc[0, :]
other_books = book_df.iloc[1:, :]
similar_books = cfilter.get_k_similar(other_books, book0)
print(similar_books[1].round(2))

Book 38    0.15
Book 53    0.13
Book 76    0.12
Book 65    0.12
Book 14    0.10
dtype: float64


## Insights and Conclusions

## References